In [153]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/sw/arcts/centos7/python3.6-anaconda/5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if __name__ == '__main__':


In [160]:
def getSynonymsByGoogle(query):
    url = "https://www.google.com/search?q=" + query
    res = requests.get(url) 
    Soup = BeautifulSoup(res.text,'html.parser') 
    links = Soup.findAll('a')
    link_img = None
    for link in links:
        if 'Images' in link.get_text():
            link_img = link.get('href')
            break
    synonyms = []
    if link_img:
        url = 'https://www.google.com'+link_img
        res = requests.get(url)
        Soup = BeautifulSoup(res.text,'html.parser') 
        span_synonyms = Soup.find_all("a",class_= "TwVfHd")
        for span in span_synonyms:
            synonyms.append(span.get_text('innerText'))
    return synonyms

def getIdxCandidates(df,keyword):
    idx_candidates = []
    for idx in range(len(df)):
        if keyword.lower() in df_qrel.loc[idx,'text'].lower():
            idx_candidates.append(idx)
    return idx_candidates

def getSynonymsToidxs(synonyms,df):
    synonymsToidxs =  { synonym: getIdxCandidates(df,synonym) for synonym in synonyms}
    idxs = []
    for synonym in synonyms:
        idxs += getIdxCandidates(df,synonym)
    return synonymsToidxs,sorted( list(set(idxs)) )

In [164]:
path_labelTemplate = '../dataset/qrel/query - template.csv'
qid = 9
query = 'data analytics' 

df_qrel = pd.read_csv(path_labelTemplate)
df_qrel['qid'] = qid
df_qrel['score'] = -1

synonyms_reference = getSynonymsByGoogle(query)
print(synonyms_reference)

['business', 'icon', 'infographic', 'machine learning', 'cloud', 'artificial intelligence', 'big data', 'marketing', 'healthcare', 'dashboard']


In [162]:
synonyms = ['machine learning','analysis','analyzed','analytics','tableau','dashboard'] + [query]
print(synonyms)

synonymsToidxs,idxs = getSynonymsToidxs(synonyms,df_qrel)
print(len(idxs),idxs)

['machine learning', 'analysis', 'analyzed', 'analytics', 'tableau', 'dashboard', 'data analytics']
39 [0, 24, 26, 31, 37, 56, 57, 85, 91, 93, 95, 122, 123, 127, 137, 143, 144, 147, 153, 154, 155, 166, 171, 180, 187, 188, 189, 216, 217, 219, 231, 233, 239, 240, 244, 264, 283, 287, 314]


In [163]:
df_qrel.iloc[idxs][['text']].style.set_properties(**{'text-align': 'left'})

,text
0,Analyzed performance on computer cache and improved it by using machine learning algorithms
24,"Built schema of services using AWS Cloudformation including log system, AWS Dashboard and Warning service, cross-account IAM role config, S3 Storage buckets and pipelines."
26,"Redesigned and developed perceptual algorithms for obstacle detection and tracking on intelligent industry truck based on OpenCV and C++. Analyzed radar point cloud data by methods of re-sampling, clustering and filtering. Quantified test results of model based on scikit-learning. Improved the tracking accuracy by 7%, meanwhile decreasing the false positive rate by 12%."
31,- Collected 2.6 billion records of service disruption information on Azure and analyzed those records to construct a Power BI dashboard to showcase recent unhealthy clusters.
37,- Analysis the data and solving the latency and accuracy issue with MATLAB and Python.
56,"● Development of courseware & editing course textbooks and lecture and assisting teachers with the basic courses of data mining, preparation, visualization, algorithm, machine learning, A/B testing and probability & Statistics."
57,"Expand the customer service management systems, analysis the usage of the product, reducing manual workload for Service data technicians and operators."
85,"• Built auto analysis data streaming pipelines to automate anomaly detection in a timely manner, saving quality assurance engineers 90% time on their daily investigations"
91,"• Analyzed clients’ requirements, designed ER models and constructed corresponding databases using SQL server including 200k records; gathered comprehensive profiles of cars and customers by SQL queries"
93,"• Deployed logistic regression, k-means clustering and cross-correlation table to conduct various analysis to find out characters of drivers, cars, roads, and environments when violations occurred"


In [103]:
idx2= [0]
idx1= []
idx0= []
for idx in idx2:
    df_qrel.at[idx, 'score'] = 2
for idx in idx1:
    df_qrel.at[idx, 'score'] = 1
for idx in idx0:
    df_qrel.at[idx, 'score'] = 0
path = "../dataset/qrel/query - %d.csv"%(qid)
df_qrel.to_csv(path)